# Preprocessing Data

This notebook is used to preprocess the data for the final project so it is ready for analysis. Each data set is preprocessed separately and then combined into one data set and saved as "full.csv". Currently the data is preprocessed and merged for the following data sets:

1. [Depth to Groundwater]()
2. [Well Elevation]()
3. [Well XY]()
4. [Daily Weather Data]()

In [1]:
import pandas as pd
import numpy as np
from datetime import date
import scipy.stats as stats

## Create definitions for the following functions:

In [2]:
"""
Function to consistently create a well ID (WID) column
"""
def create_WID(df, col_name):
    
    df['WID'] = df['GLOBAL_ID'] + '-' + df[col_name]
    return df

In [3]:
"""
Function to create a date column for the LA weather data
"""
def get_date(row):

    string = str(row['DATE'])

    year = string[:4]
    month = string[4:6]
    day = string[6:]

    date = year + '-' + month + '-' + day

    return date

def get_date(row):
    string = str(row['DATE'])
    year = string[:4]
    month = string[4:6]
    day = string[6:]
    date = year + '-' + month + '-' + day
    return date

weather['S_DATE'] = weather.apply(get_date, axis=1)
weather['DATE'] = pd.to_datetime(weather['S_DATE'])
weather =  weather[['DATE', 'ELEMENT', 'DATA_VALUE']]

## Load the data

### Geotracker Depth to Groundwater

In [4]:
# Load depth to groundwater data
geo_well = pd.read_csv('data/LosAngelesGeoWell.txt', sep='\t', encoding='unicode_escape')

# Convert date to datetime
geo_well['GW_MEAS_DATE'] = pd.to_datetime(geo_well['GW_MEAS_DATE'],format='%Y-%m-%d', errors='coerce')

# Create a mask for any rows with dates past the current date and drop them.
mask = geo_well['GW_MEAS_DATE'] >= str(date.today())
geo_well = geo_well[~mask]

# Create a mask for any rows with dates before the specified date (01-01-2000) and drop them.
mask = geo_well['GW_MEAS_DATE'] <= '01-01-2000'
geo_well = geo_well[~mask]

# Drop any rows with null values for GW_MEAS_DATE and reset the index.
geo_well.dropna(subset=['GW_MEAS_DATE'], inplace=True)
geo_well.reset_index(drop=True, inplace=True)

# Fill NaNs with 0 for the DTFPROD and DTW columns.
geo_well['DTFPROD'].fillna(0, inplace=True)
geo_well['DTW'].fillna(0, inplace=True)

# Create a new column for the total depth to groundwater.
geo_well['DEPTH'] = np.nan

# Create mask for rows with DTFPROD == 0 then sets DEPTH equal to DTW.
mask = geo_well['DTFPROD'] == 0
geo_well.loc[mask, 'DEPTH'] = geo_well['DTW']

# Create maks for rows with DTW == 0 then sets DEPTH equal to DTFPROD.
mask = geo_well['DTW'] == 0
geo_well.loc[mask, 'DEPTH'] = geo_well['DTFPROD']

# Create a mask for rows with DTFPROD != 0 and DTW != 0 then sets DEPTH equal to DTW.
mask = (geo_well['DTW'] != 0) & (geo_well['DTFPROD'] != 0)
geo_well.loc[mask, 'DEPTH'] = geo_well['DTW']

# Create mask for rows with depth of 0 and drop them.
mask = geo_well['DEPTH'] == 0
geo_well = geo_well[~mask]

# Create a WID column.
geo_well = create_WID(geo_well, 'FIELD_POINT_NAME')
geo_well['GID'] = geo_well['WID'] + ':' + geo_well['GW_MEAS_DATE'].astype(str)

c:\Users\chief\.conda\envs\geoprj\lib\site-packages\IPython\core\interactiveshell.py:3398: DtypeWarning: Columns (3,4,7,8,9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


### Geotracker Well Elevation

In [5]:
# Load the top of well elevation data.
geo_z = pd.read_csv('data/LosAngelesGeoZ.txt', sep='\t', encoding='unicode_escape', on_bad_lines='warn')

# Fill in NaN values with 0
geo_z['RISER_HT'].fillna(0, inplace=True)

# Remove invalid rows
geo_z.drop(geo_z[geo_z['RISER_HT'] == '2009-12-18'].index, inplace=True)

# Add RISER_HT to the ELEVATION column to get true well casing elevation (where depth to water is measured from).
geo_z['ELEVATION'] = geo_z['ELEVATION'] + geo_z['RISER_HT'].astype(float)

# Remove rows with invalid ELEVATION values
geo_z.dropna(subset=['ELEVATION'], inplace=True)

# Create WID column.
geo_z = create_WID(geo_z, 'FIELD_PT_NAME')

# Keep only requied columns.
geo_z = geo_z[['WID', 'ELEVATION']]

b'Skipping line 211619: expected 12 fields, saw 20\n'
c:\Users\chief\.conda\envs\geoprj\lib\site-packages\IPython\core\interactiveshell.py:3398: DtypeWarning: Columns (2,5,9,10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


### Geotracker Well XY

In [6]:
# Load the well location data.
geo_xy = pd.read_csv('data/LosAngelesGeoXY.txt', sep='\t', encoding='unicode_escape', on_bad_lines='warn')

# Create WID column.
geo_xy = create_WID(geo_xy, 'FIELD_PT_NAME')

# Create mask for rows with NaN X and Y values and drop them.
mask = (geo_xy['LATITUDE'].isna()) | (geo_xy['LONGITUDE'].isna())
geo_xy = geo_xy[~mask]

# Create mask for rows with latitude and longitude in NAD83 and keep them.
mask = geo_xy['XY_DATUM'] == 'NAD83'
geo_xy = geo_xy[mask]

# Reset the index.
geo_xy.reset_index(drop=True, inplace=True)

# Keep only required columns.
geo_xy = geo_xy[['WID', 'LONGITUDE', 'LATITUDE']]

b'Skipping line 18923: expected 13 fields, saw 14\nSkipping line 19016: expected 13 fields, saw 14\n'
b'Skipping line 200273: expected 13 fields, saw 14\nSkipping line 200274: expected 13 fields, saw 14\nSkipping line 200275: expected 13 fields, saw 14\nSkipping line 200444: expected 13 fields, saw 14\nSkipping line 200445: expected 13 fields, saw 14\nSkipping line 200446: expected 13 fields, saw 14\nSkipping line 201921: expected 13 fields, saw 14\nSkipping line 201922: expected 13 fields, saw 14\nSkipping line 202073: expected 13 fields, saw 14\nSkipping line 202074: expected 13 fields, saw 14\nSkipping line 202075: expected 13 fields, saw 14\nSkipping line 202076: expected 13 fields, saw 14\nSkipping line 202396: expected 13 fields, saw 14\nSkipping line 202660: expected 13 fields, saw 14\nSkipping line 202901: expected 13 fields, saw 14\nSkipping line 202902: expected 13 fields, saw 14\nSkipping line 205043: expected 13 fields, saw 14\nSkipping line 205044: expected 13 fields, saw 

### Los Angeles USC Weather Station Data

In [7]:
# Load the weather data.
weather = pd.read_csv('data/USW00093134.csv', engine='python')

# Create a string date column.
weather['S_DATE'] = weather.apply(get_date, axis=1)

# Fix date column using pandas' to_datetime function and the string date.
weather['DATE'] = pd.to_datetime(weather['S_DATE'])

# Keep only required columns.
weather =  weather[['DATE', 'ELEMENT', 'DATA_VALUE']]

# Create mask for the types of weather data we want to keep, i.e. precipitation, min and max temperature.
mask = (weather['ELEMENT'] == 'TMAX') | (weather['ELEMENT'] == 'TMIN') | (weather['ELEMENT'] == 'PRCP')

# Create pivot table for the weather data types, each row is a date and each column is a weather data type.
pivot = weather[mask].pivot_table(index='DATE', columns='ELEMENT', values='DATA_VALUE')

# Drop rows with NaN values and reset the index.
pivot.dropna(inplace=True)
pivot.reset_index(inplace=True)

### Merge dataframes

In [11]:
# Create Pandas dataframe for the well data by merging the depth to water and well elevation data.
geo = pd.merge(geo_well, geo_z, on='WID', how='left')

# Merge the well data with the xy data.
geo = pd.merge(geo, geo_xy, on='WID', how='left')

# Merge the well data with the weather data.
geo = pd.merge(geo, pivot, left_on='GW_MEAS_DATE', right_on='DATE', how='left')

# Drop NaNs from the ELEVATION column.
geo.dropna(subset=['ELEVATION'], inplace=True)

# Drop duplicate GID rows.
geo.drop_duplicates(subset=['GID'], inplace=True)

# Create WATER_ELEVATION column by subtracting the the depth to water from the well elevation.
geo['WATER_ELEVATION'] = geo['ELEVATION'] - geo['DEPTH']

# Drop NaNs from the WATER_ELEVATION, LATIDUTE, and LONGITUDE columns.
geo.dropna(subset=['WATER_ELEVATION','LATITUDE','LONGITUDE'], inplace=True)

# Sort values by date and then reset the index.
geo.sort_values(by=['GW_MEAS_DATE'], inplace=True)

# Create ZSCORE columns for DEPTH and WATER_ELEVATION and use them to remove outliers.
geo['ZSCORE_DEPTH'] =  stats.zscore(geo['DEPTH'])
geo['ZSCORE_WELEV'] =  stats.zscore(geo['WATER_ELEVATION'])

# Create a mask for rows with ZSCORE_DEPTH > 3 or ZSCORE_WELEV > 3 and drop them, then reset the index.
mask = (abs(geo['ZSCORE_WELEV']) >= 3)
geo = geo[~mask]
geo.reset_index(drop=True, inplace=True)

In [13]:
geo.dtypes

COUNTY                      object
GLOBAL_ID                   object
FIELD_POINT_NAME            object
STATUS                      object
GW_MEAS_DATE        datetime64[ns]
DTFPROD                    float64
DTW                        float64
RISER_HT                    object
TOT_DEPTH                   object
GW_MEAS_DESC                object
SHEEN                       object
DEPTH                      float64
WID                         object
GID                         object
ELEVATION                  float64
LONGITUDE                   object
LATITUDE                    object
DATE                datetime64[ns]
PRCP                       float64
TMAX                       float64
TMIN                       float64
WATER_ELEVATION            float64
ZSCORE_DEPTH               float64
ZSCORE_WELEV               float64
dtype: object

In [14]:
cols = ['WID', 'GW_MEAS_DATE', 'LATITUDE', 'LONGITUDE', 'DEPTH', 'ELEVATION', 'WATER_ELEVATION',  'PRCP', 'TMAX', 'TMIN']

In [15]:
geo[cols]

,WID,GW_MEAS_DATE,LATITUDE,LONGITUDE,DEPTH,ELEVATION,WATER_ELEVATION,PRCP,TMAX,TMIN
0,SLT4307472-4A,2000-01-27,34.021058,-118.451652,157.45,165.66,8.21,0.0,200.0,83.0
1,SLT4307472-115A,2000-01-27,34.018806,-118.455532,142.46,151.94,9.48,0.0,200.0,83.0
2,SL603792561-MW-14,2000-01-27,33.89102,-118.024076,60.85,76.25,15.40,0.0,200.0,83.0
3,SLT4307472-57A,2000-01-27,34.019668,-118.454001,149.01,158.18,9.17,0.0,200.0,83.0
4,SL603792561-MW-01,2000-01-27,33.890665,-118.02366,61.15,76.30,15.15,0.0,200.0,83.0
...,...,...,...,...,...,...,...,...,...,...
569613,SL204CP1893-MW3,2022-06-30,33.8817104,-118.2191509,52.41,63.96,11.55,0.0,289.0,172.0
569614,SL204CP1893-MW2,2022-06-30,33.8813501,-118.2193062,51.83,62.89,11.06,0.0,289.0,172.0
569615,SL204CP1893-MW1,2022-06-30,33.8814103,-118.2191395,52.46,63.14,10.68,0.0,289.0,172.0
569616,SL204AF1745-GMW-8,2022-06-30,33.8784256,-118.1429772,25.12,62.24,37.12,0.0,289.0,172.0


In [16]:
geo[cols].to_csv('data/full.csv', index=False)